In [22]:
# This is a project type notebook, it needs huge computational power and time for the full IMDB Dataset

In [23]:
!pip install nltk

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saims\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
import pandas as pd

In [26]:
# importing the dataframe

In [27]:
# df = pd.read_csv('IMDB Dataset.csv')

# taking only first 5000 rows
df = pd.read_csv('IMDB Dataset.csv').head(5000)

In [28]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
4995,An interesting slasher film with multiple susp...,negative
4996,i watched this series when it first came out i...,positive
4997,Once again Jet Li brings his charismatic prese...,positive
4998,"I rented this movie, after hearing Chris Gore ...",negative


In [29]:
# EDA

In [30]:
df.shape

(5000, 2)

In [31]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [32]:
df.describe()

,review,sentiment
count,5000,5000
unique,4997,2
top,Quite what the producers of this appalling ada...,negative
freq,2,2532


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     5000 non-null   object
 1   sentiment  5000 non-null   object
dtypes: object(2)
memory usage: 78.2+ KB


In [34]:
df['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [35]:
df['sentiment'].value_counts()

negative    2532
positive    2468
Name: sentiment, dtype: int64

In [36]:
# apply label encoder

In [37]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [38]:
df['sentiment'] = le.fit_transform(df['sentiment'])

In [39]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [40]:
X = df['review']
y = df['sentiment']

In [41]:
X

0       One of the other reviewers has mentioned that ...
1       A wonderful little production. <br /><br />The...
2       I thought this was a wonderful way to spend ti...
3       Basically there's a family where a little boy ...
4       Petter Mattei's "Love in the Time of Money" is...
                              ...                        
4995    An interesting slasher film with multiple susp...
4996    i watched this series when it first came out i...
4997    Once again Jet Li brings his charismatic prese...
4998    I rented this movie, after hearing Chris Gore ...
4999    This was a big disappointment for me. I think ...
Name: review, Length: 5000, dtype: object

In [42]:
y

0       1
1       1
2       1
3       0
4       1
       ..
4995    0
4996    1
4997    1
4998    0
4999    0
Name: sentiment, Length: 5000, dtype: int32

In [43]:
# removing all special and numeric character from the data and also remove stopwords and apply stemming

In [44]:
# for styling only
from tqdm import tqdm

In [45]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

In [46]:
ps = PorterStemmer()
corpus = []

for i in tqdm(range(len(X))):
#     print(i, end=', ')
    review = re.sub("[^a-zA-Z]"," ",X[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [04:40<00:00, 17.83it/s]


In [62]:
# print(corpus)

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
# cv = TfidfVectorizer(max_features=5000)
cv = TfidfVectorizer(max_features=500)
X = cv.fit_transform(corpus).toarray()

In [64]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.11308151, 0.        ,
        0.13957086],
       ...,
       [0.        , 0.        , 0.07017651, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.03484319, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [65]:
X.shape

(5000, 500)

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=81)

In [67]:
from sklearn.svm import SVC
model = SVC(C=1.0,
           kernel='linear',
           random_state=81)

In [68]:
model.fit(X_train, y_train)

SVC(kernel='linear', random_state=81)

In [69]:
model.score(X_test, y_test)

0.854

In [70]:
y_pred = model.predict(X_test)

In [71]:
# save and load the model
# import pickle

In [72]:
# pickle.dump(cv, open('movie_review_cv.pkl', 'wb'))
# pickle.dump(model, open('movie_review_model.pkl', 'wb'))

In [73]:
# save_cv = pickle.load(open('movie_review_cv.pkl', 'rb'))
# save_model = pickle.load( open('movie_review_model.pkl', 'rb'))

In [78]:
# function for testing

def test_model(sentence):
#     sen = save_cv.transform([sentence]).toarray()
    sen = cv.transform([sentence]).toarray()
#     res = save_model.predict(sen)[0]
    res = model.predict(sen)[0]
    #     print(res)
    if res == 1:
        print('positive')
    else:
        print('negative')

In [79]:
sentence = 'i love amber heard'
test_model(sentence)

positive


In [80]:
sentence = 'i hate gigi hadid'
test_model(sentence)

negative


In [81]:
# print(set(stopwords.words("english")))

In [82]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.86      0.85      0.86       507
           1       0.85      0.85      0.85       493

    accuracy                           0.85      1000
   macro avg       0.85      0.85      0.85      1000
weighted avg       0.85      0.85      0.85      1000

